In [ ]:
import sys

In [ ]:
import intake
import pandas as pd
import geopandas as gpd
from calitp_data_analysis import geography_utils
from siuba import *
import shapely

In [ ]:
from shared_utils import catalog_utils, rt_dates
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [ ]:
from segment_speed_utils import helpers

# Exploratory Analysis

Start working on how to identify and count "border zones" where transit runs along Census Tract boundaries and VRH/VRM should be split accordingly (avoiding double-counting or arbitrary allocation to only one border tract)

In [ ]:
catalog = intake.open_catalog("*.yml")

In [ ]:
analysis_date = rt_dates.DATES['feb2025']

In [ ]:
def read_census_tracts(
    crs: str = geography_utils.CA_NAD83Albers_m
) -> gpd.GeoDataFrame:
    census_tracts = (
        catalog.calenviroscreen_lehd_by_tract.read()
        .to_crs(crs)
        [["Tract", "pop_sq_mi", "geometry"]]
    ).rename(columns={'Tract':'tract'})
    return census_tracts

In [ ]:
tracts = read_census_tracts()

In [ ]:
stops = helpers.import_scheduled_stops(analysis_date, columns=['feed_key', 'stop_id', 'geometry'])

In [ ]:
stops

In [ ]:
tracts_with_stops = tracts.sjoin(stops, predicate='contains', how='inner') >> distinct(_.tract, _.geometry)

In [ ]:
tracts_with_stops

In [ ]:
shapes = helpers.import_scheduled_shapes(analysis_date)

In [ ]:
# shapes.explore()

In [ ]:
BORDER_BUFFER_METERS = 35

In [ ]:
tracts_with_stops.crs

In [ ]:
tracts_with_stops.geometry = tracts_with_stops.buffer(BORDER_BUFFER_METERS)

In [ ]:
borders = gpd.overlay(tracts_with_stops, tracts_with_stops)

In [ ]:
borders

In [ ]:
def intersection_hash(row):
    '''
    Get unique hash of intersection zones.
    No need to keep both t1 x t2 and t2 x t1
    '''
    t1 = int(row.tract_1[2:]) #  drop state code
    t2 = int(row.tract_2[2:])
    row_tracts = [t1, t2]
    row_tracts.sort() #  modifies inplace
    return hash(tuple(row_tracts))

In [ ]:
borders = borders[borders['tract_1'] != borders['tract_2']]

In [ ]:
borders['intersection_hash'] = borders.apply(intersection_hash, axis=1)

In [ ]:
borders = borders >> distinct(_.intersection_hash, _keep_all=True)

In [ ]:
test = borders.head(4000)
# test = borders

In [ ]:
gdf = shapes.clip(test) #  TODO try sjoin instead -- ideally distances contained in one border zone

In [ ]:
# test2 = gdf.head(2000)
test2 = gdf

In [ ]:
test2 = test2.explode() #  split multilines before dropping short (crossing) segments

In [ ]:
test2 = test2[test2.geometry.length > BORDER_BUFFER_METERS*4]

In [ ]:
test2.geometry.length

In [ ]:
m = test.explore(tiles='CartoDB Positron')

In [ ]:
test2.shape_array_key.isna().all()

In [ ]:
# test2.explore(color='orange', m=m, tiles='CartoDB Positron')

## interpolation points

In [ ]:
%%time

test2 = test.overlay(test, how='intersection')

test2 = test2.query('intersection_hash_1 != intersection_hash_2')

test2.head(5000).explore()

test

In [ ]:
shapes

In [ ]:
helpers.import_scheduled_trips?

In [ ]:
cols = ['gtfs_dataset_key', 'name', 'trip_id', 'shape_id', 'shape_array_key', 'route_id', 'route_key', 'direction_id', 'route_short_name', 'trip_instance_key', 'feed_key']

In [ ]:
trips = helpers.import_scheduled_trips(analysis_date, columns=cols)

In [ ]:
trips = trips.dropna(subset=['shape_id'])

In [ ]:
act_6 = trips.query('name.str.contains("AC Transit") & shape_id == "shp-6-03"')

In [ ]:
act_6.shape_array_key.unique()

In [ ]:
act_6_shape = shapes.query('shape_array_key.isin(@act_6.shape_array_key)')

In [ ]:
act_6_shape.explore()

In [ ]:
act_6_shape.explore(color='orange', m=m, tiles='CartoDB Positron')

In [ ]:
def overlay_to_borders(
    shape_gdf: gpd.GeoDataFrame,
    border_gdf: gpd.GeoDataFrame,
    sensitivity_dist: int = BORDER_BUFFER_METERS * 4
                 ):
    '''
    
    '''
    overlaid = shape_gdf.overlay(border_gdf, how='intersection')
    overlaid = overlaid.query('geometry.length > @sensitivity_dist')
    return overlaid

In [ ]:
overlay_to_borders(act_6_shape, borders)

In [ ]:
border_overlaid = overlay_to_borders(act_6_shape, borders)

In [ ]:
border_overlaid.explore(column='intersection_hash', tiles='CartoDB Positron', categorical=True)

In [ ]:
not_border = act_6_shape.overlay(border_overlaid, how='difference')

In [ ]:
not_border

In [ ]:
def overlay_to_tracts(
    shape_gdf_no_border: gpd.GeoDataFrame,
    tract_gdf: gpd.GeoDataFrame,
                 ):
    '''
    
    '''
    tract_gdf = tract_gdf[['tract', 'geometry']]
    return shape_gdf_no_border.overlay(tract_gdf, how='intersection')

In [ ]:
tract_overlaid = overlay_to_tracts(not_border, tracts)

In [ ]:
tract_overlaid.explore(column='tract', tiles='CartoDB Positron', categorical=True)

In [ ]:
tracts_and_borders = pd.concat([tract_overlaid, border_overlaid]).reset_index(drop=True)

In [ ]:
tracts_and_borders = tracts_and_borders.assign(border = ~tracts_and_borders.tract_2.isna())

In [ ]:
tracts_and_borders = tracts_and_borders.assign(
    start = tracts_and_borders.geometry.apply(lambda x: shapely.Point(x.coords[0])),
    end = tracts_and_borders.geometry.apply(lambda x: shapely.Point(x.coords[-1])))

In [ ]:
tracts_and_borders.reset_index().explore(column='border', tiles='CartoDB Positron', categorical=True)

In [ ]:
tracts_and_borders

In [ ]:
tracts_and_borders.to_parquet(f'test_tracts_borders_{analysis_date}.parquet')